## Install Dependencies
Only needed on first run, to ensure you have the correct versions of required libraries.

In [ ]:
%pip install --force-reinstall -r requirements.txt

## Imports
Imports libraries

In [21]:
import os
import pandas as pd
from Modules.rf_wrapper import RF_Model, Preprocess, Evaluate

## Files & Directories
Sets up variables used when performing file I/O.

In [22]:
# set up directories
data_dir = os.getcwd() + r'/Data'
model_dir = os.getcwd() + r'/Models'
predict_dir = os.getcwd() + r'/Predictions'

# dataset names
data_fnames = ["demo"]

## Initialise Model
Creates an instance of RF_Model, and loads the model and scaler from files.

In [23]:
rf = RF_Model()
if rf.LoadGridSearch(f'{model_dir}/random_forest.joblib'):
    print("Successfully loaded model.")

if rf.LoadScaler(f'{model_dir}/std_scaler.joblib'):
    print("Successfully loaded scaler.")

Successfully loaded model.
Successfully loaded scaler.


## Make Predictions
Iterates through each dataset:
* Loads the dataset into a Pandas dataframe.
* Performs preprocessing on the dataset.
* Makes predictions on the processed data.
* Prints model performance.
* Exports predictions to a zipped .csv file.

If an exception is encountered at any point, stops the loop early.

In [24]:
for name in data_fnames:
    try:
        df = pd.read_csv(f'{data_dir}/{name}.zip', compression='zip')
        df_p = Preprocess(df)
    except Exception as e1:
        print(f"ERROR: an exception was thrown while loading or preprocessing the {name} dataset!\n", repr(e1))
        break
    else:
        try:
            pred = rf.Predict(df_p)
            pred_accuracy, pred_f1 = Evaluate(df_p['IsDDoS'].to_numpy(), pred)
            print(f"{name}:\n Accuracy: {pred_accuracy}\n F1 Score: {pred_f1}")
        except Exception as e2:
            print(f"ERROR: an exception was thrown while making predictions on the {name} dataset!\n", repr(e2))
            break
        else:
            try:
                pred_fname = f"{name}_{pd.Timestamp.today(tz='Australia/Perth').strftime('%d-%m-%Y')}"
                df['IsDDoS_Truth'] = df_p['IsDDoS']
                df['IsDDoS_Predict'] = pred
                df.to_csv(f"{predict_dir}/{pred_fname}.zip", compression={"method": "zip", "archive_name": f"{pred_fname}.csv"}, index=False)
            except Exception as e3:
                print(f"ERROR: an exception was thrown while exporting {name} predictions!\n", repr(e3))
                break

demo:
 Accuracy: 0.4642857142857143
 F1 Score: 0.5976082740788622
